----
----
# **Análisis en General**
----
----
## Carga de tabla y preparación

In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from my_functions import *

In [2]:
table = pd.read_csv('my_tables/table_hist.csv')
table['fecha'] = pd.to_datetime(table['fecha'])
table['evento_p'] = table['evento_p'].fillna('') 
table['evento_t'] = table['evento_t'].fillna('')
table.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       7468 non-null   datetime64[ns]
 1   dia_semana  7468 non-null   object        
 2   semana      7468 non-null   object        
 3   dolar_b     7468 non-null   float64       
 4   dolar_o     7468 non-null   float64       
 5   brecha_s    7468 non-null   float64       
 6   brecha      7468 non-null   float64       
 7   db_var      7468 non-null   float64       
 8   do_var      7468 non-null   float64       
 9   brecha_var  7468 non-null   float64       
 10  evento_p    7468 non-null   object        
 11  evento_t    7468 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 700.2+ KB


----
## 5- Con la info histórica del valor del dólar y del blue, realizar un análisis exploratorio. Cruzar la data con sucesos importantes a nivel político-económico y graficar mes a mes.

In [3]:
# Creamos una tabla de eventos
eventos = table[table['evento_t'] != '']
eventos.head()

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
54,2002-04-27,Saturday,2002-16,3.123333,3.04,0.083333,2.741228,-2.699896,-0.327869,-2.504674,Roberto Lavagna,econ
119,2002-07-01,Monday,2002-26,3.830000,3.79,0.040000,1.055409,-0.777202,-0.087873,-0.702060,Aldo Pignanelli,bcra
273,2002-12-02,Monday,2002-48,3.620000,3.59,0.030000,0.835655,-0.366972,-0.092764,-0.277518,Fin del Corralito,misc
282,2002-12-11,Wednesday,2002-49,3.530000,3.51,0.020000,0.569801,-0.563380,-0.566572,0.003228,Alfonso Prat-Gay,bcra
283,2002-12-11,Wednesday,2002-49,3.530000,3.51,0.020000,0.569801,-0.563380,-0.566572,0.003228,Alfonso Prat-Gay,bcra


In [4]:
px.bar(x = eventos['dia_semana'].value_counts().index, 
       y = eventos['dia_semana'].value_counts().values/len(eventos), 
       labels={'y':'Porcentaje','x':'Día de la semana'}, 
       title='Ocurrencias de eventos')

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['dolar_b'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['dolar_b'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['db_var'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['db_var'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Variación del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['dolar_o'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['dolar_o'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table['fecha'], y = table['do_var'], mode='lines'))
fig.add_trace(go.Scatter(x = eventos['fecha'], y = eventos['do_var'], mode='markers', name='Eventos ocurridos'))
fig.update_layout(
    title='Variación del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

**Obsevaciones**
- Los eventos ocurren los **lunes** cerca de un 30% de las veces.
- Un evento de **julio 2019** pareció elevar el **valor de ambos dólares**.
- Un evento del **diciembre 2015** coincide con una la mayor **variación del dólar oficial**.

In [9]:
eventos[eventos['fecha'] == '2015-12-17']

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
5042,2015-12-17,Thursday,2015-50,14.36,13.76,0.6,4.360465,-1.441318,39.979654,-43.85927,Levantamiento del Cepo al Dolar,misc


In [10]:
eventos[eventos['fecha'] == '2019-07-01']

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
6335,2019-07-01,Monday,2019-26,43.45,42.14,1.31,3.108685,-0.267789,-0.244615,-0.023959,Hernán Lacunza,trea


1. El **Levantamiento del Cepo al Dolar** está lugado con la **mayor variación al dolar oficial** en la historia.
2. El **nombramiento de Hernán Lacuzna** como ministro de Hacienda pudo ser detonante de aumento en el valor de ambos dólares de Júlio 2019.
----
## 6- Implementar una regresión lineal (una para cada tipo de dólar) para predecir el valor del dólar en:
* 3 meses
* 6 meses
* 12 meses

### **Modelo de regresión para predicción del valor de Dólar Blue**

In [11]:
# Pasamos las fechas a números
table_reg_0 = table.copy()
table_reg_0['fecha_o'] = table_reg_0['fecha'].map(lambda x : dt.datetime.toordinal(x))
table_reg_0 = table_reg_0[['fecha','fecha_o', 'dolar_b']]
table_reg_0.head()

,fecha,fecha_o,dolar_b
0,2002-03-04,730913,2.12
1,2002-03-05,730914,2.12
2,2002-03-06,730915,2.14
3,2002-03-07,730916,2.22
4,2002-03-08,730917,2.27


In [12]:
X = table_reg_0['fecha_o'].values.reshape(-1,1)
y = table_reg_0['dolar_b'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

dolar_b_lr0 = LinearRegression(fit_intercept=True)
dolar_b_lr0.fit(X_train, y_train)

y_predict_train = dolar_b_lr0.predict(X_train)
y_predict = dolar_b_lr0.predict(X_test)

print('Error en datos de train:', mean_squared_error(y_train, y_predict_train))
print('Error en datos de test:', mean_squared_error(y_test, y_predict))

fig = go.Figure()
fig.add_trace(go.Scatter(x = X_test[:,0], y = y_test, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = X_test[:,0], y = y_predict, mode='lines', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')


Error en datos de train: 1639.3907821807325
Error en datos de test: 1622.717990852996


Una hipótesis es que usar valores en la **fecha a partir de 1ro de mayo del 2016** como mi nuevo 0, se podrá ajustar más a **una relación cuadrática**. Por lo que eso es lo que voy a probar enseguida: 

In [13]:
day_0_b = dt.datetime.strptime('2016-05-01', '%Y-%m-%d')                              # El 1/5/2016 será mi nuevo 0
table_reg_1 = table.copy()
table_reg_1['fecha_o'] = table_reg_1['fecha'].map(lambda x : date_to_x(x, day_0_b)) # Preparo las x's cuadradas
table_reg_1.dropna(inplace = True)                                                # Me quedo con las fechas arriva de mi 0
table_reg_1 = table_reg_1[['fecha', 'fecha_o', 'dolar_b']]
table_reg_1.head()

,fecha,fecha_o,dolar_b
5178,2016-05-01,0.0,14.666667
5179,2016-05-02,1.0,14.670000
5180,2016-05-03,4.0,14.660000
5181,2016-05-04,9.0,14.670000
5182,2016-05-05,16.0,14.580000


In [14]:
X = table_reg_1['fecha_o'].values.reshape(-1,1)
y = table_reg_1['dolar_b'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 0)

dolar_b_lr1 = LinearRegression(fit_intercept=True)
dolar_b_lr1.fit(X_train, y_train)

y_predict_train = dolar_b_lr1.predict(X_train)
y_predict = dolar_b_lr1.predict(X_test)
print('Error en datos de train:', mean_squared_error(y_train, y_predict_train))
print('Error en datos de test:', mean_squared_error(y_test, y_predict))
fig = go.Figure()
fechas = [x_to_date(x, day_0_b) for x in X_test[:,0]]
fig.add_trace(go.Scatter(x = fechas, y = y_test, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = fechas, y = y_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

Error en datos de train: 202.98311298283014
Error en datos de test: 239.4057920932903


**¡El error de los datos pasó de 1622.71 a 239.40!**<br>
Procedo a hacer las predicciones:

In [15]:
today = dt.datetime.today()
fechas_a_predecir = np.array([])
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=90)) # 3 meses
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=180)) # 6 meses
fechas_a_predecir = np.append(fechas_a_predecir, today + dt.timedelta(days=360)) # 12 meses
x_b_predict = np.array([date_to_x(x, day_0_b) for x in fechas_a_predecir])
y_b_predict = dolar_b_lr1.predict(x_b_predict.reshape(-1,1))
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_reg_1['fecha'], y = table_reg_1['dolar_b'], mode='lines', name = 'Actual_data'))
fig.add_trace(go.Scatter(x = fechas_a_predecir, y = y_b_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [16]:
print("Predicciones del valor del dólar blue:")
for index in range(len(fechas_a_predecir)):
    print('Predicción para {} : {}'.format(dt.datetime.strftime(fechas_a_predecir[index], '%Y-%m-%d'), y_b_predict[index]))

Predicciones del valor del dólar blue:
Predicción para 2022-11-03 : 265.3684780197765
Predicción para 2023-02-01 : 285.5156762422131
Predicción para 2023-07-31 : 328.0560444129565


### **Modelo de regresión para predicción del valor de Dólar Oficial**

In [17]:
table_reg_2 = table.copy()
table_reg_2['fecha_o'] = table_reg_2['fecha'].map(lambda x : dt.datetime.toordinal(x))
table_reg_2 = table_reg_2[['fecha', 'fecha_o', 'dolar_o']]
table_reg_2.head()

,fecha,fecha_o,dolar_o
0,2002-03-04,730913,2.01
1,2002-03-05,730914,1.99
2,2002-03-06,730915,2.05
3,2002-03-07,730916,2.14
4,2002-03-08,730917,2.20


In [18]:
X = table_reg_2['fecha_o'].values.reshape(-1,1)
y = table_reg_2['dolar_o'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

dolar_o_lr0 = LinearRegression(fit_intercept=True)
dolar_o_lr0.fit(X_train, y_train)

y_predict_train = dolar_o_lr0.predict(X_train)
y_predict = dolar_o_lr0.predict(X_test)
print('Error en datos de train:', mean_squared_error(y_train, y_predict_train))
print('Error en datos de test:', mean_squared_error(y_test, y_predict))
fig = go.Figure()
fig.add_trace(go.Scatter(x = X_test[:,0], y = y_test, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = X_test[:,0], y = y_predict, mode='lines', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

Error en datos de train: 392.79571960937074
Error en datos de test: 397.68603041244484


Intentaré usar el mismo modelo que el dolar blue (cuadrático) usando como mi 0 a la fecha 1/10/2013

In [19]:
day_0_o = dt.datetime.strptime('2013-10-01', '%Y-%m-%d')                              # El 1/5/2016 será mi nuevo 0
table_reg_3 = table.copy()
table_reg_3['fecha_o'] = table_reg_3['fecha'].map(lambda x : date_to_x(x, day_0_o)) # Preparo las x's cuadradas
table_reg_3.dropna(inplace = True)                                                # Me quedo con las fechas arriva de mi 0
table_reg_3 = table_reg_3[['fecha', 'fecha_o', 'dolar_o']]
table_reg_3.head()

,fecha,fecha_o,dolar_o
4232,2013-10-01,0.0,5.800000
4233,2013-10-02,1.0,5.800000
4234,2013-10-03,4.0,5.810000
4235,2013-10-04,9.0,5.810000
4236,2013-10-05,16.0,5.813333


In [20]:
X = table_reg_3['fecha_o'].values.reshape(-1,1)
y = table_reg_3['dolar_o'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 0)

dolar_o_lr1 = LinearRegression(fit_intercept=True)
dolar_o_lr1.fit(X_train, y_train)

y_predict_train = dolar_o_lr1.predict(X_train)
y_predict = dolar_o_lr1.predict(X_test)
print('Error en datos de train:', mean_squared_error(y_train, y_predict_train))
print('Error en datos de test:', mean_squared_error(y_test, y_predict))
fig = go.Figure()
fechas = [x_to_date(x, day_0_o) for x in X_test[:,0]]
fig.add_trace(go.Scatter(x = fechas, y = y_test, mode='markers', name = 'Test data'))
fig.add_trace(go.Scatter(x = fechas, y = y_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

Error en datos de train: 26.166806086826586
Error en datos de test: 26.0009560390255


**¡El error de los datos pasó de 397.68 a 26.00!**<br>
Procedo a hacer las predicciones:

In [21]:
x_o_predict = np.array([date_to_x(x, day_0_o) for x in fechas_a_predecir])
y_o_predict = dolar_o_lr1.predict(x_o_predict.reshape(-1,1))
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_reg_3['fecha'], y = table_reg_3['dolar_o'], mode='lines', name = 'Actual_data'))
fig.add_trace(go.Scatter(x = fechas_a_predecir, y = y_o_predict, mode='markers', name='Prediction'))
fig.update_layout(
    title='Valor del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

In [22]:
print("Predicciones del valor del dólar oficial:")
for index in range(len(fechas_a_predecir)):
    print('Predicción para {} : {}'.format(dt.datetime.strftime(fechas_a_predecir[index], '%Y-%m-%d'), y_o_predict[index]))

Predicciones del valor del dólar oficial:
Predicción para 2022-11-03 : 126.23078384751207
Predicción para 2023-02-01 : 133.0905378618852
Predicción para 2023-07-31 : 147.36045705968957


----
## 7- Bonus opcional: Realizar una calculadora de predicción de aumento del dólar

In [23]:
def cal_predict_dolar(fecha, tipo = 'blue'):
    fecha = dt.datetime.strptime(fecha, '%Y-%m-%d')
    if tipo == 'blue':
        day_0 = dt.datetime.strptime('2016-05-01', '%Y-%m-%d') 
        x = date_to_x(fecha, day_0)
        y = dolar_b_lr1.predict(np.array([[x]]))
    elif tipo == 'oficial':
        day_0 = dt.datetime.strptime('2013-10-01', '%Y-%m-%d') 
        x = date_to_x(fecha, day_0)
        y = dolar_o_lr1.predict(np.array([[x]]))
    else:
        return None
    return y[0]

In [24]:
print(cal_predict_dolar('2022-11-03', 'oficial'))
print(cal_predict_dolar('2022-11-03', 'blue'))

126.23078384751207
265.3684780197765


----
## 8-  Mejor momento para comprar dolár oficial y venderlo a dolár blue
En los últimos 4 años

In [25]:
# Creamos tabla de los últimos 4 años:
first_day = dt.datetime.today()-dt.timedelta(days = 1459) # 4 años
table_4_anios = table[table['fecha'] > first_day][['fecha', 'brecha_s', 'brecha']]
table_4_anios.head()

,fecha,brecha_s,brecha
6008,2018-08-08,0.280000,1.017442
6009,2018-08-09,-0.020000,-0.071505
6010,2018-08-10,-0.200000,-0.684932
6011,2018-08-11,-0.236667,-0.803077
6012,2018-08-12,-0.273333,-0.919076


Usando la brecha en soles:

In [26]:
table_4_anios.sort_values('brecha_s', ascending = False).head(1)

,fecha,brecha_s,brecha
7454,2022-07-22,208.24,160.480888


Usando la brecha en porcentaje:

In [27]:
table_4_anios.sort_values('brecha', ascending = False).head(1)

,fecha,brecha_s,brecha
7454,2022-07-22,208.24,160.480888


**Conclusión**: Resulta coincidir en que el **22 de Julio de 2022** fue el **mejor momento de los últimos 4 años para comprar Oficial y vender Blue** tanto en porcentaje como en soles. <br>
* Mejor en porcentaje: El retorno de inversión es el mayor (160%)
* Mejor en soles: La cantidad de soles ganados por transacción es la máxima ($208.4 soles)

---------------